# V1

The goal of this is to get a basic version up and running

Approach 1
- Infer dialog act
- Look only at action names at first (maybe descriptions), if ambiguous, then look at action signatures, if ambiguous look at pre/eff, if still ambiguous ask for clarification. 
    - Need to be more general here. The imperative could be specifying a sequence of actions (not sure DIARC can handle this anyway), or an action that cannot occur unless another action is performed first --> constant interaction with planner is needed. Sometimes action imperatives are even just goal directives in disguise (go to the door --> no goToDoor action, but at(door) is a possible state. How to convert "go to" to "be at". "put two blue blocks on top of the red one". 


idea
- Check if action names exist, if not then reword the utterance as a goal, and see if it can be handled that way
- If only one name exists, then check action signature and types
    - if that works, then check if precons are met, and if so, perform action
    - If not, then set precons as goal, plan and generate action sequence, and then also perform target action
- If multiple matches, then check to see the "best" action signature
    - Select best action signature by 

Other things:
- Get a pddl-based planner involved --> external calls 

Variations
- human involvement could include providing some set of conditions that specify several states, or they provide a a group of actions (a miniplan) and refer to that by a name
- human could provide action performance constraints - do X, but don't touch Y.

- Challenges with reference resolution .


In [14]:
# Spot testing
data = [{"utterance": "Pick up the ball", "act": "INSTRUCT"},
       {"utterance": "Get the circuit breaker on the work area", "act": "INSTRUCT"},
       {"utterance": "First raise your arms", "act": "INSTRUCT"},
       {"utterance": "Dempster, who do you trust?", "act": "QUESTION"},
       {"utterance": "Dempster, do you see an object?", "act": "QUESTION"},
       {"utterance": "The area behind you is safe", "act": "STATEMENT"},
       {"utterance": "The object in front of you is a ball", "act": "STATEMENT"},
       {"utterance": "It uses a medical caddy", "act": "STATEMENT"}]

intents = ["INSTRUCT", "QUESTION", "STATEMENT"]


In [15]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [16]:
#llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)
llm = OpenAI(temperature=0.0)

In [17]:
# CHAINS

template_dialog_act_classifier= """
Decide whether the utterance below from a speaker to a listener is an INSTRUCT, QUESTION or STATEMENT. 
An INSTRUCT is an imperative statement or a request by the speaker to have the listener do an action.
A QUESTION is a query or request from a speaker for more information from the listener about the listeners knowledge, beliefs or perceptions
A STATEMENT is a statement of fact or opinion that the speaker conveys to a listener. 

utterance: \n{utterance}\n
act:
"""

prompt_dialog_act_classifier = PromptTemplate(
    input_variables=["utterance"],
    template=template_dialog_act_classifier
)

chain_dialog_act_classifier = LLMChain(llm=llm, prompt=prompt_dialog_act_classifier)



In [18]:
def parse(utterance):
    print(f"Utterance: {utterance}")
    act = chain_dialog_act_classifier.run(utterance=utterance)
    
    output = {'utterance': utterance,
              'act':act,}
    return output


In [19]:
for item in data:
    output = parse(item['utterance'])
    print(output)
    assert output['act'] == item['act']
    

Utterance: Pick up the ball
{'utterance': 'Pick up the ball', 'act': 'INSTRUCT'}
Utterance: Get the circuit breaker on the work area
{'utterance': 'Get the circuit breaker on the work area', 'act': 'INSTRUCT'}
Utterance: First raise your arms
{'utterance': 'First raise your arms', 'act': 'INSTRUCT'}
Utterance: Dempster, who do you trust?
{'utterance': 'Dempster, who do you trust?', 'act': 'QUESTION'}
Utterance: Dempster, do you see an object?
{'utterance': 'Dempster, do you see an object?', 'act': 'QUESTION'}
Utterance: The area behind you is safe
{'utterance': 'The area behind you is safe', 'act': 'STATEMENT'}
Utterance: The object in front of you is a ball
{'utterance': 'The object in front of you is a ball', 'act': 'STATEMENT'}
Utterance: It uses a medical caddy
{'utterance': 'It uses a medical caddy', 'act': 'STATEMENT'}
